In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('ModifiedData_Recommendation')
# Slicing the data since the data was too big and the kernel had been crashing.
df = df.iloc[:10000, :]
df.drop(df.columns[0], axis=1, inplace=True)
print(df.columns)
df.head()

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')


,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [6]:
df.user_id.unique().shape[0]

9366

In [7]:
"""
Finding the number of unique users and items which will be used for creating
a user-item matrix
"""
num_users = df.user_id.unique().shape[0]
num_items = df.business_id.unique().shape[0]

""" Sanity Check """
print(num_users, num_items)

9366 4618


In [4]:
""" Creating a unique numeric id for every user_id """

unique_users = df.user_id.unique()
user_id_dict = {}

for number,user in enumerate(unique_users):
    user_id_dict[user] = [number, user]

# Each element in user_id_dict now looks like this:  TQY1CtRrKuNJS4dSDLbjdw [274, 'TQY1CtRrKuNJS4dSDLbjdw']
# The key is the  user_id and the value is a list containing the numeric id we've generated and the actual user_id

In [5]:
""" Run this code if you want to check the contents of user_id_dict """
# for key,value in user_id_dict.items():
#     print(key,value)

' Run this code if you want to check the contents of user_id_dict '

In [6]:
""" Creating a unique numeric id for every business_id """
unique_items = df.business_id.unique()
business_id_dict = {}
business_id_dict_for_output = {}

for number,item in enumerate(unique_items):
    business_id_dict[item] = [number, item]
    business_id_dict_for_output[number] = item

# Each element in business_id_dict now looks like this:  vkLCTKcM_gw7XhhJsM8AsA [359, 'vkLCTKcM_gw7XhhJsM8AsA']
# The key is the  business_id and the value is a list containing the numeric id that we've 
# generated and the actual business_id

In [7]:
""" Run this code if you want to check the contents of business_id_dict """
# for key,value in business_id_dict.items():
#     print(key,value)

' Run this code if you want to check the contents of business_id_dict '

In [8]:
""" Adding the numeric user_id's to the original dataset """
numeric_user_id_list = []
for each_user_id in df.user_id:
    numeric_user_id_list.append(user_id_dict[each_user_id][0])
df['numeric_user_id'] = numeric_user_id_list

""" Adding the numeric business_id's to the original dataset """
numeric_business_id_list = []
for each_business_id in df.business_id:
    numeric_business_id_list.append(business_id_dict[each_business_id][0])
df['numeric_business_id'] = numeric_business_id_list

In [9]:
""" Sanity Check """
df.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,numeric_user_id,numeric_business_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA,0,0
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg,1,1
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw,2,2
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg,3,3
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ,4,4
5,eU_713ec6fTGNO4BegRaww,0,2013-01-20 13:25:59,0,fdiNeiN_hoCxCMy2wTRW9g,4,I'll be the first to admit that I was not exci...,0,w31MKYsNFMrjhWxxAb5wIw,5,5
6,3fw2X5bZYeW9xCz_zGhOHg,5,2016-05-07 01:21:02,4,G7XHMxG0bx9oBJNECG4IFg,3,Tracy dessert had a big name in Hong Kong and ...,5,jlu4CztcSxrKx56ba1a5AQ,6,6
7,zvO-PJCpNk4fgAVUnExYAA,1,2010-10-05 19:12:35,1,8e9HxxLjjqc9ez5ezzN7iQ,1,This place has gone down hill. Clearly they h...,3,d6xvYpyzcfbF_AZ8vMB7QA,7,7
8,b2jN2mm9Wf3RcrZCgfo1cg,0,2015-01-18 14:04:18,0,qrffudO73zsslZbe8B9D3Q,2,I was really looking forward to visiting after...,1,sG_h0dIzTKWa3Q6fmb4u-g,8,8
9,oxwGyA17NL6c5t1Etg5WgQ,1,2012-02-29 21:52:43,0,RS_GTIT6836bCaPy637kNQ,3,It's a giant Best Buy with 66 registers. I do...,1,nMeCE5-xsdleyxYuNZ_7rA,9,9


In [10]:
""" 
Creating a matrix of 0's with rows= num_users and columns = num_items
This will be used to map the ratings given by every unique user to every 
unique item in the dataset 
(user-item matrix)
"""
data_matrix = np.zeros((num_users, num_items))
data_matrix.shape

(9366, 4618)

In [11]:
""" 
Creating a new dataframe with the fields necessary to fill inside the data_matrix that we've created above 
"""
cols = ['numeric_user_id', 'numeric_business_id', 'stars']
df_new = pd.DataFrame()
df_new = df[cols]
df_new.head()

,numeric_user_id,numeric_business_id,stars
0,0,0,1
1,1,1,5
2,2,2,5
3,3,3,5
4,4,4,1


In [12]:
df_new.head()

,numeric_user_id,numeric_business_id,stars
0,0,0,1
1,1,1,5
2,2,2,5
3,3,3,5
4,4,4,1


In [13]:
""" Creating a user-item matrix which can be used to calculate the similarity between users and items"""
for each_line in df_new.itertuples():
    data_matrix[each_line[1], each_line[2]] = each_line[3]

In [14]:
""" We will use cosine similarity to calculate the similarities """
""" We can use pairwise_distanc function from sklearn to calculate the cosine similarity """

from sklearn.metrics.pairwise import pairwise_distances

# Calculating similarities between users for user-based collaborative filtering
user_similarity = pairwise_distances(data_matrix, metric='cosine')

# Calculating similarities between items for item-based collaborative filtering
item_similarity = pairwise_distances(data_matrix.T, metric='cosine')

In [15]:
""" Defining a function to make predictions based on the similarities that we have calculated above """

def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        # To make sure that mean_user_rating has the same format as ratings, we'll use np.newaxis.
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    
    return pred

In [16]:
"""
The following function makes the predictions for a given user_id and returns a list of the top 10 predictions.
"""

def get_predictions(user, type_of_filtering='user'):
    """
    Params: user_id for whom you want to find predictions and the type of filtering that you'd want to perform
    Return: List of top 10 predictions
    """
    mapped_user_id = user_id_dict[user][0]
    if type_of_filtering == 'user':
        predictions_for_user = predict(data_matrix, user_similarity, type='user')[mapped_user_id, :]
    elif type_of_filtering == 'item':
        # Calling the predict function that we've defined above and parsing the results
        predictions_for_user = predict(data_matrix, item_similarity, type='item')[mapped_user_id, :]
    temp_dict = {}
    for index, business_to_predict in enumerate(predictions_for_user):
        temp_dict[business_to_predict] = index
    top_sorted_predictions = sorted(temp_dict.keys(), reverse=True)[:10]
    temp_numeric_id_list = []
    for each_prediction in top_sorted_predictions:
        temp_numeric_id_list.append(temp_dict[each_prediction])
    final_prediction_list = []
    for single_prediction in temp_numeric_id_list:
        final_prediction_list.append(business_id_dict_for_output[single_prediction])
    print('The following the top 10 predictions for the given user:\n')
    return final_prediction_list

In [17]:
""" Finally, making the predictions for any user by providing the user_id """

# To make predictions with user based filtering, pass the keyword argument: type_of_filtering='user'

get_predictions('hG7b0MtEbXx5QzbzE6C_VA', type_of_filtering='user')

The following the top 10 predictions for the given user:



['Wxxvi3LZbHNIDwJ-ZimtnA',
 'faPVqws-x-5k2CQKDNtHxw',
 'ujHiaprwCQ5ewziu0Vi9rw',
 'XXW_OFaYQkkGOGniujZFHg',
 'SVGApDPNdpFlEjwRQThCxA',
 'Xg5qEQiB-7L6kGJ5F4K3bQ',
 'HhVmDybpU7L50Kb5A0jXTg',
 'yNPh5SO-7wr8HPpVCDPbXQ',
 'pH0BLkL4cbxKzu471VZnuA',
 'wUKzaS1MHg94RGM6z8u9mw']

In [18]:
# To make predictions with item based filtering, pass the keyword argument: type_of_filtering='item'

get_predictions('hG7b0MtEbXx5QzbzE6C_VA', type_of_filtering='item')

The following the top 10 predictions for the given user:



['T2jKa-mP-zLjKX6oYRaU5Q',
 'gkxmQi2cipbysXFgT_w_jA',
 'lkM72Y21bjBqUGaW7iL7tQ',
 'h8bJb1dIwPmUGiqstCy7lw',
 'sqdhck-Ad8UiBTVRbhdepw',
 'LPnGdeEJyMMO02e02PqZBA',
 'Xbd_sL81qS3eMqISjHrW4g',
 'QoRL0_AwNV-GkHFe8tvp2g',
 'V1wBDq0zqq2dlyQvzD5PaA',
 'GWO87Y-IqL54_Ijx6hTYAQ']

In [19]:
"""
Reference: https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/
"""

'\nReference: https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/\n'